# Träna RL för att balansera Cartpole

Den här anteckningsboken är en del av [AI för nybörjare-kursplanen](http://aka.ms/ai-beginners). Den har inspirerats av [officiell PyTorch-handledning](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) och [denna Cartpole PyTorch-implementation](https://github.com/yc930401/Actor-Critic-pytorch).

I det här exemplet kommer vi att använda RL för att träna en modell att balansera en stång på en vagn som kan röra sig åt vänster och höger på en horisontell skala. Vi kommer att använda [OpenAI Gym](https://www.gymlibrary.ml/) för att simulera stången.

> **Note**: Du kan köra kodexemplen i denna lektion lokalt (t.ex. från Visual Studio Code), i vilket fall simuleringen öppnas i ett nytt fönster. När du kör koden online kan du behöva göra vissa justeringar i koden, som beskrivs [här](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Vi börjar med att säkerställa att Gym är installerat:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Nu ska vi skapa CartPole-miljön och se hur vi kan arbeta med den. En miljö har följande egenskaper:

* **Action space** är mängden möjliga åtgärder som vi kan utföra vid varje steg i simuleringen
* **Observation space** är utrymmet för observationer som vi kan göra


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Låt oss se hur simuleringen fungerar. Följande loop kör simuleringen tills `env.step` inte returnerar termineringsflaggan `done`. Vi kommer att välja åtgärder slumpmässigt med hjälp av `env.action_space.sample()`, vilket innebär att experimentet förmodligen kommer att misslyckas väldigt snabbt (CartPole-miljön avslutas när hastigheten på CartPole, dess position eller vinkel är utanför vissa gränser).

> Simuleringen kommer att öppnas i ett nytt fönster. Du kan köra koden flera gånger och se hur den beter sig.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Du kan märka att observationerna innehåller 4 siffror. Dessa är:
- Vagnens position
- Vagnens hastighet
- Stolpens vinkel
- Stolpens rotationshastighet

`rew` är belöningen vi får vid varje steg. Du kan se att i CartPole-miljön får du 1 poäng för varje simuleringssteg, och målet är att maximera den totala belöningen, det vill säga tiden som CartPole kan balansera utan att falla.

Under förstärkningsinlärning är vårt mål att träna en **policy** $\pi$, som för varje tillstånd $s$ talar om för oss vilken åtgärd $a$ vi ska ta, så i princip $a = \pi(s)$.

Om du vill ha en probabilistisk lösning kan du tänka på policyn som att den returnerar en uppsättning sannolikheter för varje åtgärd, det vill säga $\pi(a|s)$ skulle innebära sannolikheten att vi ska ta åtgärd $a$ i tillstånd $s$.

## Policy Gradient-metoden

I den enklaste RL-algoritmen, kallad **Policy Gradient**, kommer vi att träna ett neuralt nätverk för att förutsäga nästa åtgärd.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Vi kommer att träna nätverket genom att köra många experiment och uppdatera vårt nätverk efter varje körning. Låt oss definiera en funktion som kommer att köra experimentet och returnera resultaten (så kallad **spår**) - alla tillstånd, handlingar (och deras rekommenderade sannolikheter) och belöningar:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Du kan köra ett avsnitt med ett otränat nätverk och observera att den totala belöningen (AKA längden på avsnittet) är mycket låg:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

En av de knepiga aspekterna med policy gradient-algoritmen är att använda **diskonterade belöningar**. Idén är att vi beräknar vektorn av totala belöningar vid varje steg i spelet, och under denna process diskonterar vi de tidiga belöningarna med någon koefficient $gamma$. Vi normaliserar också den resulterande vektorn, eftersom vi kommer att använda den som vikt för att påverka vår träning:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Nu kör vi träningen! Vi kommer att köra 300 episoder, och vid varje episod gör vi följande:

1. Kör experimentet och samla in spårdata.
2. Beräkna skillnaden (`gradients`) mellan de utförda handlingarna och de förutspådda sannolikheterna. Ju mindre skillnaden är, desto säkrare är vi på att vi har valt rätt handling.
3. Beräkna diskonterade belöningar och multiplicera gradienterna med de diskonterade belöningarna - detta säkerställer att steg med högre belöningar påverkar slutresultatet mer än de med lägre belöningar.
4. Förväntade målhandlingar för vårt neurala nätverk kommer delvis att tas från de förutspådda sannolikheterna under körningen och delvis från de beräknade gradienterna. Vi använder parametern `alpha` för att bestämma i vilken utsträckning gradienter och belöningar tas i beaktande - detta kallas *inlärningshastighet* för förstärkningsalgoritmen.
5. Slutligen tränar vi vårt nätverk på tillstånd och förväntade handlingar och upprepar processen.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Nu låt oss köra avsnittet med rendering för att se resultatet:


In [ ]:
_ = run_episode(render=True)

Förhoppningsvis kan du nu se att stången kan balansera ganska bra!

## Actor-Critic Modell

Actor-Critic-modellen är en vidareutveckling av policy gradients, där vi bygger ett neuralt nätverk för att lära oss både policyn och de uppskattade belöningarna. Nätverket kommer att ha två utgångar (eller så kan du se det som två separata nätverk):
* **Actor** kommer att rekommendera vilken åtgärd som ska vidtas genom att ge oss sannolikhetsfördelningen för tillståndet, precis som i policy gradient-modellen.
* **Critic** skulle uppskatta vad belöningen skulle bli från dessa åtgärder. Den returnerar totalt uppskattade belöningar i framtiden vid det givna tillståndet.

Låt oss definiera en sådan modell:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Vi skulle behöva göra små ändringar i våra `discounted_rewards` och `run_episode` funktioner:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Nu kommer vi att köra huvudträningsloopen. Vi kommer att använda en manuell nätverksträningsprocess genom att beräkna korrekta förlustfunktioner och uppdatera nätverksparametrar:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Slutsats

Vi har sett två RL-algoritmer i denna demonstration: enkel policy gradient och den mer sofistikerade actor-critic. Du kan se att dessa algoritmer arbetar med abstrakta begrepp som tillstånd, handling och belöning - vilket gör att de kan tillämpas på mycket olika miljöer.

Förstärkningsinlärning gör det möjligt för oss att lära oss den bästa strategin för att lösa problemet enbart genom att titta på den slutliga belöningen. Det faktum att vi inte behöver märkta dataset gör att vi kan upprepa simuleringar många gånger för att optimera våra modeller. Dock finns det fortfarande många utmaningar inom RL, som du kan lära dig mer om om du väljer att fokusera mer på detta intressanta område inom AI.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiska översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
